Thanks to RareSkill's awesome ZK book I've acquired enough knowledge to implement Groth16 in python.<br>Though I don't expect this to be easy, I'll leverage my giga brain and exceptional resilience to apply what I've learned to this complex and interesting project.

### Quick introduction to Groth16
Groth16 is a magic algorithm to validate that a statement is true without revealing any information about the inputs.<br>
For example, imagine that you want to prove to someone that you know a secret number that satisfies a specific equation without revealing the number itself; <br>the Groth16 algorithm could do just that.
It is known for its efficiency and small proof size, though it comes with one compromise: The need of a trusted setup.

### Polynomial Secret Proof PoC 
To dive into the practical implementation of Groth16, I'll start with a foundational example that showcases its core strengths.<br> 
I'm going to implement a system that proves knowledge of a secret value that satisfies a polynomial equation:<br> something like proving **I know a secret {x,y,z} such that $5x² + xy + 4z³ = 1059$**, without ever revealing what s actually is. <BR><BR> The prover (likely the user) must provide the correct ${x,y,x}$ values which in our case are: $x = 13$,  $y = 14$,  $z = 2$ though they will stay private and the verifier will not have access to these raw values.

#### Steps and required setup overview (ordered):
- Define the R1CS constraint system based on our polynomial equation
- Convert R1CS -> QAP
- Implement the trusted setup phase to generate proving and verification keys
- Build the prover that creates ZK proofs using the secret witness
- Build the verifier that validates proofs without learning the secret
- Test the complete workflow with our polynomial example

---

### Defining the R1CS constraints

Lets say we are proving our earlier  equation $1059 = 5x² + xy + 4z³$, we want to prove that we know the solution ($x = 13$,  $y = 14$,  $z = 2$).<BR><BR>
We want to only have a single multiplication:<BR>

-  $v_1 = xx$ <BR>
-  $v_2 = 5v_1$
-  $v_3 = zz$
-  $v_4 = 4z * v_3$<BR><BR><BR> So we are left with a single mutliplication:<BR><BR>
$1059 = v_2 + xy + v_4$<BR><BR>
Simplified:<BR><BR>$1059 - v_2 - v_4 = xy$

<BR><BR>
we define the witness vector with all variables, including a constant $1$ as the first variable:<BR><BR>
let $out = 1059$<BR><BR>
let $w$ be the witness vector<BR><B>

$w = [1,out,x,y,z,v_1,v_2,v_3,v_4]$<BR>
$w = [1,1059,13,14,2,169,845,4,32]$
<BR><BR><B>then define $O=L*R$, with our 5 constraints<BR>

$L = \begin{bmatrix}
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
5 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 4 & 0 & 0 & 0 & 0 \\
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
\end{bmatrix}$

$R = \begin{bmatrix}
0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 1 & 0 & 0 & 0 & 0 & 0 \\
\end{bmatrix}$

$O = \begin{bmatrix}
0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 & 0 \\
0 & 0 & 0 & 0 & 0 & 0 & 0 & 0 & 1 \\
0 & 1 & 0 & 0 & 0 & 0 & -1 & 0 & -1 \\
\end{bmatrix}$

Lets verify the circuit is correct with code

In [14]:
from py_ecc.optimized_bn128 import curve_order
import galois
import numpy as np

# define finite field on the bn128 EC (commonly used by Ethereum)
GF = galois.GF(curve_order) # = 21888242871839275222246405745257275088548364400416034343698204186575808495617

# witness vector
w = GF([1,1059,13,14,2,169,845,4,32])

# [1,out,x,y,z,v_1,v_2,v_3,v_4]
L = GF(np.array([
    [0, 0, 1, 0, 0, 0, 0, 0, 0],
    [5, 0, 0, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 4, 0, 0, 0, 0],
    [0, 0, 1, 0, 0, 0, 0, 0, 0],
]))

R = GF(np.array([
    [0, 0, 1, 0, 0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 1, 0, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 1, 0, 0, 0, 0, 0],
]))

O = GF(np.array([
    [0, 0, 0, 0, 0, 1, 0, 0, 0],
    [0, 0, 0, 0, 0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0, 0, 0, 1, 0],
    [0, 0, 0, 0, 0, 0, 0, 0, 1],
    [0, 1, 0, 0, 0, 0, curve_order-1, 0, curve_order-1], # since we are working on a finite field i.e. -1 mod 17 = 16
]))

result = np.matmul(O, w) == np.multiply(np.matmul(L, w), np.matmul(R, w))

assert result.all(), "incorrect, should be equal"
print("Ow = Lw * Rw: " , result.all())



Ow = Lw * Rw:  True


#### As expected **it works**! If we changed even a single number it would obviously fail.

## R1CS -> QAP


*One Step Closer To **The Powers Of TAU!!** (exceptional naming sense).*<BR><BR> Now it is time to turn the R1CS to a QAP for the verifier, leveraging the Lagrange interpolations to represent the R1CS as polynomials for each column of all of the matrices,<BR> and the Schwartz-Zippel Lemma in order to achieve succintness. This helps groth16 be incredibly more efficient and massively reducing the proof size.


### First: transorm R1CS into polynomials

Here is the code for it, looping through each column:

In [21]:
# we have 5 constraints so interpolate 5 points
xs = GF(np.array([1,2,3,4,5]))

def interpolate_col(c):
    return galois.lagrange_poly(xs, c)

U_polys = []
V_polys = []
W_polys = []

# interpolate each column
for col_index in range(9):
    # extract column
    L_col = L[:, col_index]
    R_col = R[:, col_index]
    O_col = O[:, col_index]

    # get Lagrange'd
    U_poly = interpolate_col(L_col)
    V_poly = interpolate_col(R_col)
    W_poly = interpolate_col(O_col)

    U_polys.append(U_poly)
    V_polys.append(V_poly)
    W_polys.append(W_poly)

print("interplolated all columns\n")
print("Result:\n\n")
print(U_polys)
print(V_polys)
print(W_polys)


interplolated all columns

Result:


[Poly(18240202393199396018538671454381062573790303667013361953081836822146507079680x^4 + 3648040478639879203707734290876212514758060733402672390616367364429301415947x^3 + 3648040478639879203707734290876212514758060733402672390616367364429301415887x^2 + 18240202393199396018538671454381062573790303667013361953081836822146507079770x + 21888242871839275222246405745257275088548364400416034343698204186575808495567, GF(21888242871839275222246405745257275088548364400416034343698204186575808495617)), Poly(0, GF(21888242871839275222246405745257275088548364400416034343698204186575808495617)), Poly(20064222632519335620392538599819168831169334033714698148390020504361157787649x^4 + 21888242871839275222246405745257275088548364400416034343698204186575808495616x^3 + 12768141675239577212977070018066743801653212566909353367157285775502554955781x^2 + 10944121435919637611123202872628637544274182200208017171849102093287904247800x + 6, GF(218882428718392752222464057452572

### Completing the QAP equation
As of now we have computed this: <BR><BR>
$\underbrace{\sum_{i=1}^{5} a_i u_i(x)}_\text{U polys from L}  \underbrace{\sum_{i=1}^5 a_i v_i(x)}_\text{ V polys from R} = \underbrace{\sum_{i=1}^{5} a_i w_i(x)}_\text{ W polys from O}$
The verification is not succinct just yet, reason being that the verifier verifies all constraint points individually: 
- Check $U(1) \cdot V(1) = W(1)$
- $U(2) \cdot V(2) = W(2)$

And so on... we'd need 5 separate checks in our case.

#### Enabling succinctness
To enable succintness we need to introduce a set of two polynomial $h(x)$ and $t(x)$. $t(x)$ being $t(x) = (x - 1)(x - 2)...(x - 5)$<BR><BR>
knowing this we can compute $h(x)$ which formula is this: $h(x) = \frac{u(x)v(x) – w(x)}{t(x)}$. <BR><BR>
Now the verifier only needs ONE check, invoking the Powers of Tau...